In [ ]:
import sys
!pip install datasets
!pip install spacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz
import os
os.chdir('/content/drive/MyDrive/internal/mpng_cust/conditions/')
import utlts
from utlts import *

In [ ]:
%%time
import pickle
data = pd.read_csv('/content/test.csv')
data['SOURCE_CODE'] = data['SOURCE_CODE'].apply(lambda x: clean_input(str(x)))
dat = DatasetDict({'test':Dataset.from_dict({'label': data.TARGET_CONCEPT_NAME,'text':data.SOURCE_CODE})})

In [15]:
%%time
preds = []
for datum in tqdm(dat['test']['text']):
    src_indexes = torch.tensor(text_transform_sou(datum)).unsqueeze(0).to(device)
    mapped_sentence_ids = translate_seq_beam_search(transformer_model, src_indexes, k=1, device=device, max_len=50)
    mapped_sentence_ids = sorted(mapped_sentence_ids, key= lambda x: x[1], reverse=True)
    translation = [[tar_vocab.get_itos()[i] for i in mapped_sentence[0]] for mapped_sentence in mapped_sentence_ids]
    preds.append(' '.join([i for i in translation[0] if i not in ['<sos>', '<eos>', '<pad>', '<unk>']]))

100%|██████████| 17424/17424 [11:35<00:00, 25.06it/s]

CPU times: user 11min 25s, sys: 3.03 s, total: 11min 28s
Wall time: 11min 35s


In [ ]:
correct = 0
for i, j in zip(preds, data.TARGET_CONCEPT_NAME.tolist()):
  if i.lower() == j.lower(): correct +=1

In [22]:
correct / data.shape[0]

0.6000918273645547

In [23]:
for n in range(20):
    print(f"Example {n+1}\n")
    ex = dat['test'][random.randint(0, len(dat['test']))]
    sentence = ex['text']
    src_indexes = torch.tensor(text_transform_sou(sentence)).unsqueeze(0).to(device)
    k = 3
    translated_sentence_ids = translate_seq_beam_search(transformer_model, src_indexes, k=k, device=device, max_len=30)
    translated_sentence_ids = sorted(translated_sentence_ids, key= lambda x: x[1], reverse=True)
    translations = [[tar_vocab.get_itos()[i] for i in translated_sentence[0]] for translated_sentence in translated_sentence_ids]
    print(f"Source : {ex['text']}")
    print(f"SNOMED : {ex['label']}\n")
    print(f"Mapping generated:\n")
    for i in range(k):
        for w in translations[i]:
            if w in ['<sos>', '<eos>', '<pad>', '<unk>']:
                continue
            print(w, end=" ")
        print()
    print("---------------------------------------------------------------------\n")
del src_indexes, ex, sentence, translated_sentence_ids, translations
torch.cuda.empty_cache()
_ = gc.collect()

Example 1

Source : cervix cancer
SNOMED : primary malignant neoplasm of uterine cervix

Mapping generated:

malignant tumor of cervix 
primary malignant neoplasm of uterine cervix 
neoplasm of uterine cervix 
---------------------------------------------------------------------

Example 2

Source : bursitis hip bilateral
SNOMED : bursitis

Mapping generated:

bursitis of hip 
bursitis 
bursitis of bilateral hip 
---------------------------------------------------------------------

Example 3

Source : fatigue
SNOMED : fatigue

Mapping generated:

fatigue 
fatigability 
fatigue due to fatigue 
---------------------------------------------------------------------

Example 4

Source : wound healed well
SNOMED : wound

Mapping generated:

wound healed 
wound 
wound wound healed 
---------------------------------------------------------------------

Example 5

Source : metastatic left eye melanoma
SNOMED : malignant melanoma

Mapping generated:

malignant melanoma 
neoplasm of eye proper 
